In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# path='/content/drive/MyDrive/NLP_ASSIGNMENTS/'
path=''

In [3]:
import numpy as np
import math
import random
import _pickle as pickle
from scipy import spatial
from gensim.models import Word2Vec
import logging

FUNCTION TO CREATE AN DICTIONARY WHERE KEYS ARE WORDS IN VOCAB AND VALUE IS ITS POSITION

In [4]:
def get_order(vocbD):
    order={}
    pos=0
    for i in vocbD:
        order[i]=pos
        pos+=1
    return order

FUNCTION FOR COALS ALGORITHM

In [5]:
def coals(text,order,show_steps=False): 
    window=[]
    w=np.zeros((len(order),len(order)))   #initializing the matrix with zeros
    for tx in text:                    # variable text is a list where each element of that list is another list, containing words, which represents a single sentance
      n=len(tx)                         # i.e. [['I', 'am', 'Samiparna'],['This','is','NLP-Assignment']]
      for i in range(n):
        if tx[i] not in order:          #ignoring if the current word is not in the vocabulary
          continue
        window=[]     
        for j in range(i+1,i+5):        
            if j<n:
                window.append(tx[j])      #creating the window of word
        for j in range(len(window)):      
            if window[j] not in order:    #ignoring word in current window which is not in vocab
              continue
            x=order[tx[i]]
            y=order[window[j]]
            w[x,y]+=(4-j)
            w[y,x]+=(4-j)
  
    t=np.sum(w)                         #sum of all the values in matrix
    w1=np.zeros_like(w)                 #initializing new matrix to store the resultant matrix into
    swaj=np.sum(w,axis=1)                 #sum of each coloumn
    swaj=np.reshape(swaj,(len(swaj),1))   # reshaping the coloumnwise sum into a (n x 1) matrix for later matrix operation
    swib=np.sum(w,axis=0)                 #sum of each rows
    swib=np.reshape(swib,(1,len(swib)))   # reshaping the coloumnwise sum into a (1 x n) matrix for later matrix operation
    w1=((t*w)-np.dot(swaj,swib))/(np.sqrt(np.dot(swaj*(t-swaj),swib*(t-swib))))     #calculating the resultant matrix
    if show_steps== True:
        print(np.round(w1,3))
    w1[w1<=0]=0         #discarding negative values
    if show_steps==True:
      print('discarded negative values')
      print(w1)
    w1=np.sqrt(w1)        #taking square root of each value to get final matrix
    if show_steps==True:
      print('final matrix calculated')
      print(w1)
    return w1

In [6]:
def reduce_matrix(mat,row):
    u, s, vt = np.linalg.svd(mat)       #SVD to get decomposition
    s=np.diag(s)                        #diagonalizing the mtrix
    row=min(row,u.shape[1])             #taking minimum of new row count and original row count
    u1=u[:,:row]                    # only keeping first row many values         
    s1=s[:row,:row]
    vt1=vt[0:row]
    return u1,s1,vt1

In [7]:
def findSimilar(w,n):
  vec=u[vocaborder[w]]
  vecList=[]
  for i in range(u.shape[0]):
    if vocab[i]==w:
      continue
    vecList.append([vocab[i],1-spatial.distance.cosine(vec,u[i])])
  vecList=sorted(vecList,key= lambda x: x[1],reverse=True)
  if n<=len(vecList):
    return vecList[:n]
  else:
    return vecList

In [8]:
f=open(path+'preprocessed.txt','r')     #opening preprocessed text file
txt=f.read().split('.')              #seperating each sentance
f.close()
words=[r.split() for r in txt]      #creating the list of all sentences
del txt         #freeing up memory 

In [9]:
LOWEST_FREQ=252  #this is the lowest frequency calculated by some trial and error for which the vocabulary will consist more than 14,000 many unique words

In [10]:

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = Word2Vec(vector_size=50, window=5, workers=5,min_count=LOWEST_FREQ) #initializing the word2vec model
model.build_vocab(words)                    #building vocabulary by the word2vec model

#As the vocabulary of the model and COALs must be the same and words in the vocab also must be in same order 
#so i let the word2vec model create the vocabulary and then i have used it in COALS

2022-05-19 16:29:42,156 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=50, alpha=0.025>', 'datetime': '2022-05-19T16:29:42.156527', 'gensim': '4.2.0', 'python': '3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}
2022-05-19 16:29:42,164 : INFO : collecting all words and their counts
2022-05-19 16:29:42,164 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-19 16:29:42,183 : INFO : PROGRESS: at sentence #10000, processed 87832 words, keeping 8093 word types
2022-05-19 16:29:42,214 : INFO : PROGRESS: at sentence #20000, processed 176328 words, keeping 11176 word types
2022-05-19 16:29:42,240 : INFO : PROGRESS: at sentence #30000, processed 254227 words, keeping 12684 word types
2022-05-19 16:29:42,259 : INFO : PROGRESS: at sentence #40000, processed 334796 words, keeping 13954 word types
2022-05-19 16:29:42,306 : INFO : PROGRESS: at sen

In [11]:
vocab=model.wv.index_to_key.copy()      #copying vocab created by word2vec into our vocab 

In [12]:
vocaborder=get_order(vocab)             #creating index for each words in the same order they appear in the vocab
mat=coals(words,vocaborder)             #generating the Matrix
print(mat)

[[0.07804498 0.         0.05301434 ... 0.         0.         0.        ]
 [0.         0.16032428 0.         ... 0.00161377 0.01558942 0.        ]
 [0.05301434 0.         0.04504776 ... 0.         0.         0.        ]
 ...
 [0.         0.00161377 0.         ... 0.         0.         0.        ]
 [0.         0.01558942 0.         ... 0.         0.04332151 0.        ]
 [0.         0.         0.         ... 0.         0.         0.07167917]]


In [13]:
u,s,vt=reduce_matrix(mat,50)       #reducing the matrix so that each vector be of size 50
print('u= ', u)
print('s= ' ,s)
print('vt= ',vt)


u=  [[-2.70304649e-02 -6.48143659e-03  7.56854723e-02 ... -2.76603452e-02
   1.14564482e-02 -4.29161288e-03]
 [-2.94964212e-02 -7.21800877e-02 -1.78192392e-03 ... -1.53470512e-02
   3.12315460e-02 -3.78892672e-02]
 [-1.96019666e-02  1.14722601e-02 -6.43857880e-04 ...  1.69474242e-02
  -1.77436263e-03  1.30473645e-02]
 ...
 [-4.47417753e-03  5.46645365e-04 -1.79501475e-03 ...  2.44216728e-03
   1.68804007e-03  2.76715761e-03]
 [-4.21370522e-03 -2.47908226e-03 -2.45283782e-03 ... -1.77669367e-03
   5.76391367e-05  1.88991345e-03]
 [-2.43019376e-03  8.30170453e-04  9.89465318e-04 ...  8.10757842e-04
   4.75684932e-03 -6.49307972e-03]]
s=  [[22.16894267  0.          0.         ...  0.          0.
   0.        ]
 [ 0.         11.57518756  0.         ...  0.          0.
   0.        ]
 [ 0.          0.          9.1364924  ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  2.72882378  0.
   0.        ]
 [ 0.          0.          0.         ...  0.          2.70

In [14]:
f=open(path+'datas2','wb')              
pickle.dump((words,vocab,mat,u,s,vt),f)     # writing necessary datas into file for any future usage or debuging
f.close()

In [15]:
#This portion is for reading the dumped datas if needed for skipping all the rework

# import _pickle as pickle
# f=open(path+'datas2','rb')
# (words,vocab,mat,u,s,vt)=pickle.loads(f.read())
# f.close()

In [16]:
model.wv.vectors=u.astype('float32')                #initializing input embedding matrix
model.train(words,total_examples=model.corpus_count,epochs=20)  
model.save('trainedModel.bin')                      #saving the trained model

2022-05-19 17:06:18,060 : INFO : Word2Vec lifecycle event {'msg': 'training model with 5 workers on 14183 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2022-05-19T17:06:18.059233', 'gensim': '4.2.0', 'python': '3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'train'}
2022-05-19 17:06:19,166 : INFO : EPOCH 0 - PROGRESS: at 1.21% examples, 1254698 words/s, in_qsize 10, out_qsize 0
2022-05-19 17:06:20,182 : INFO : EPOCH 0 - PROGRESS: at 2.31% examples, 1209703 words/s, in_qsize 9, out_qsize 0
2022-05-19 17:06:21,187 : INFO : EPOCH 0 - PROGRESS: at 3.44% examples, 1192357 words/s, in_qsize 9, out_qsize 0
2022-05-19 17:06:22,191 : INFO : EPOCH 0 - PROGRESS: at 4.54% examples, 1198055 words/s, in_qsize 10, out_qsize 1
2022-05-19 17:06:23,192 : INFO : EPOCH 0 - PROGRESS: at 5.71% examples, 1200079 words/s, in_qsize 9, out_qsize 0
2022-05-19 17:

In [17]:
#set of covid related words. 

wo=['reported','droplet','infected','testing','death','sanitizing','transmission','strain','vaccination','severe']

In [18]:
#This is just by the COLAS word vector
print('finding similar words by word vector created using COALS:-')
for w in wo:
  print('words similar to',w,': ')
  print(findSimilar(w,5))

finding similar words by word vector created using COALS:-
words similar to reported : 
[['documented', 0.6629614402526602], ['dong', 0.6309350264866685], ['prevalent', 0.6154757752167503], ['accounted', 0.5987263798660152], ['guan', 0.5985136764869945]]
words similar to droplet : 
[['aerosol', 0.9006862753253261], ['expelled', 0.8769155912381651], ['exhaled', 0.8587249159217564], ['respirable', 0.8436455618276459], ['airborne', 0.8161084888634285]]
words similar to infected : 
[['uninfected', 0.8482218860070992], ['susceptible', 0.8317127118904099], ['noninfected', 0.8261000326764297], ['latently', 0.8065367124066386], ['asymptomatically', 0.8011148226075564]]
words similar to testing : 
[['confirmatory', 0.8448376027807426], ['serological', 0.8308487922282459], ['serologic', 0.8307394857001996], ['confirmation', 0.8150945383727773], ['serology', 0.8100657404833058]]
words similar to death : 
[['fatality', 0.8002591162303756], ['mortality', 0.7479909693342771], ['toll', 0.734190590454

In [19]:
#This is the result of trained word2vec model where the  input embedding matrix was initialized with the word vectors estimated using COALS-SVD
print('finding similar words by word2vec model:-')
for w in wo:
  print('words similar to',w,': ')
  print(model.wv.most_similar(w,topn=5))

finding similar words by word2vec model:-
words similar to reported : 
[('documented', 0.846659243106842), ('found', 0.834536075592041), ('noted', 0.8068538308143616), ('noticed', 0.7252920866012573), ('demonstrated', 0.7234799861907959)]
words similar to droplet : 
[('aerosol', 0.7865779995918274), ('particle', 0.7667319178581238), ('exhaled', 0.7431405186653137), ('expelled', 0.7383527755737305), ('airborne', 0.7164188623428345)]
words similar to infected : 
[('susceptible', 0.7853173017501831), ('recovered', 0.7805265784263611), ('exposed', 0.7414881587028503), ('uninfected', 0.7297021746635437), ('reinfected', 0.7079185843467712)]
words similar to testing : 
[('screening', 0.7945518493652344), ('test', 0.7639535069465637), ('confirmation', 0.7529978156089783), ('serology', 0.7440984845161438), ('retesting', 0.7350926399230957)]
words similar to death : 
[('fatality', 0.8017532229423523), ('mortality', 0.6977206468582153), ('hospitalization', 0.6550416946411133), ('hospitalisation',